Created Tue Mar 29 13:27:47 2022 ; Last updated 08/17/2022

@author: **Michelle M. Fink
         Colorado Natural Heritage Program, Colorado State University**

Purpose: Automate Least-Cost Corridor analysis using ArcPro arcpy

----
License: GNU General Public License version 3.
 This script is free software: you can redistribute it and/or modify
 it under the terms of the GNU General Public License as published by
 the Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 This program is distributed in the hope that it will be useful,
 but WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 GNU General Public License for more details.

 You should have received a copy of the GNU General Public License
 along with this program.  If not, see https://www.gnu.org/licenses/

# Set up

In [1]:
import os
import numpy as np
from osgeo import osr, gdal
import arcpy
from arcpy import env
from arcpy import da
from arcpy import sa
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

## environment and variables

In [4]:
#Set environments
strWorkspace = r"M:\GIS_Projects\JeffCo"
env.workspace = strWorkspace
env.scratchWorkspace = r"M:\GIS_Projects\JeffCo\Default.gdb"
permRas = r"M:\GIS_Projects\JeffCo\JC_perm_macro3.tif" #permeability raster
env.snapRaster = permRas
env.extent = permRas
env.overwriteOutput = True
env.cellSize = "MINOF"
env.compression = "LZW"
env.pyramid = "NONE"
env.rasterStatistics = "STATISTICS 1 1"
env.parallelProcessingFactor = "100%"
env.resamplingMethod = "BILINEAR"

#Set parameters
usestones = True #stones need to be in the fromCores and/or toCores shapefile with attribute "Patch" == 'step'
costname = os.path.join(strWorkspace, "JC_cost_macro3.tif")
#Cores are expected to be polygon shapefiles with attributes "CoreID", "ToID", "Patch" ("Patch" == 'core')
fromCores = r"M:\GIS_Projects\JeffCo\from_cores.shp"
toCores = r"M:\GIS_Projects\JeffCo\toCores_and_steps.shp"
elevRas = r"M:\GIS_Projects\JeffCo\JeffCo_10mNED_utm.tif"
vf = "VF.txt" #Vertical factor for least-cost analysis
spp = "MVerts"
maxdist = 60000

**A Note on maximum distance**  
A maximum distance (`maxdist`) of 60 cost-distance km was initially used, based on seasonal movement estimates for elk[^1]. No corridors from urban parcels were possible with this maximum distance, however, so an additional run was made with `maxdist = 200000`.  

**Vertical Factor**  
Contents of the file, VF.txt:  

|   .    |   .    |
| ------ | ------ |
| -90 | -1 |
| -80 | 0.2 |
| -70 | 0.3 |
| -60 | 0.4 |
| -50 | 0.5 |
| -40 | 0.6 |
| -30 | 0.7 |
| -20 | 0.8 |
| -10 | 0.9 |
| 0 | 1 |
| 10 | 1 |
| 20 | 1 |
| 30 | 1 |
| 40 | 1 |
| 50 | 1 |
| 60 | 1 |
| 70 | 1 |
| 80 | 1 |
| 90 | -1 |


## Functions

In [5]:
def raster2array(ras_name):
    """
    Convert a geoTIFF to a 2D numpy array.
    You could also do this with arcpy.RasterToNumPyArray
    ras_name: string, path/name of raster
    """
    ras_tif = gdal.Open(ras_name)
    ras_band = ras_tif.GetRasterBand(1)
    ras_data = ras_band.ReadAsArray()
    ras_tif = None
    ras_ary = ras_data
    return ras_ary

def perm_to_cost(perm, stones=None):
    """
    Convert a 0-10 Permeability raster into a Cost raster
    perm: string, path/name of permeability raster
    stones: string, path/name of stepping-stone raster, if any
    """
    if stones is None:
        cost = (sa.Raster(perm) / 10) ** -2
    else:
        rvsdperm = sa.Con(sa.IsNull(stones), sa.Raster(perm), sa.Raster(perm) + 1)
        cost = (rvsdperm / 10) ** -2
    return cost

def tryslice(rasCorridor):
    """
    Equation to use to attempt to auto-delimit least-cost corridors
    rasCorridor: string, path/name of generated cost accumulation raster
    """
    ary = raster2array(rasCorridor)
    mskary = np.ma.masked_less_equal(ary, 0)
    arymin = np.ma.min(mskary)
    cor1 = (1.0195 * arymin) + 114.44
    cor0 = cor1 - 200
    cor2 = cor1 + 200

    print("Minimum: " + str(arymin))
    print([cor0, cor1, cor2])
    return [cor0, cor1, cor2]

def create_corridors(fromRas, toRas, species, maxdist):
    """
    Get unique pairs of Cores and generate corridors for each pair
    fromRas: string, path/name of the originating Core areas raster
    toRas: string, path/name of the destination Core areas raster
    species: string, name to use as prefix of generated corridor files
    maxdist: integer, maximum distance to use, in cost-meters
    """
    corID = "_to".join([os.path.basename(fromRas).strip(species+".tif"), 
                         os.path.basename(toRas).strip(species+".tif")])
    outName1 = species + "_Accum" + corID + ".tif"
    outName2 = species + "_Corridors" + corID + ".tif"
    if os.path.isfile(os.path.join(strWorkspace, outName1)):
        accumRas = sa.Raster(os.path.join(strWorkspace, outName1))
    else:
        accumRas = sa.Raster(fromRas) + sa.Raster(toRas)
        outRas1 = sa.Con(accumRas <= maxdist, accumRas, 0)
        outRas1.save(os.path.join(strWorkspace, outName1))
        
    print("    " + outName1 + " saved")
    thresh = tryslice(os.path.join(strWorkspace, outName1))
    try:
        outRas2 = sa.Con((accumRas > 0) & (accumRas <= thresh[0]), 0,
                         sa.Con((accumRas > thresh[0]) & (accumRas <= thresh[1]), 1,
                                sa.Con((accumRas > thresh[1]) & (accumRas <= thresh[2]), 2)))
        outRas2.save(os.path.join(strWorkspace, outName2))
        print("    " + outName2 + " saved")
        
    except TypeError:
        print("Cannot create " + outName2)

**A Note on the tryslice equation**  
This equation is based on research by the author of the relationship between the minimum value present in the least-cost accumulation raster and the value threshold needed to create a truly least-cost corridor. No single set of parameters works in all situations, unfortunately, but this was the optimal equation for this project.  

![Corridor Equation](file:///M:/GIS_Projects/JeffCo/CorridorEquation.png)

# Main

In [6]:
arcpy.MakeFeatureLayer_management(fromCores, "from_core")
arcpy.MakeFeatureLayer_management(toCores, "to_core")

if usestones:
    if not os.path.isfile(os.path.join(strWorkspace, "stoneRas.tif")):
        print("Make Stepping Stone raster")
        fromsteps = arcpy.SelectLayerByAttribute_management("from_core", "NEW_SELECTION", "Patch = 'step'")
        tosteps = arcpy.SelectLayerByAttribute_management("to_core", "NEW_SELECTION", "Patch = 'step'")
        arcpy.Merge_management([fromsteps, tosteps], "tempstone")
        arcpy.conversion.FeatureToRaster("tempstone", "Patch", "stoneRas.tif", cell_size=10)
    stone = "stoneRas.tif"
else:
    stone = None
    
if os.path.isfile(costname):
    print("Found cost layer, using " + costname)
    costRas = costname
else:
    print("Converting Permeability to Cost")
    costRas = perm_to_cost(permRas, stones=stone)
    costRas.save(costname)

Make Stepping Stone raster
Found cost layer, using M:\GIS_Projects\JeffCo\JC_cost_macro3.tif


In [7]:
#Iterate through core areas and get corridors for each
print("On to all the Path Distances")

arcpy.MakeTableView_management(fromCores, "from_tbl")

with da.SearchCursor("from_tbl", ["CoreID", "ToID"]) as curSrch:
    for row in curSrch:
        #make a Path Distance raster from each Core
        valstr = str(row[0])
        constr = str(row[1])
        print("    Creating Path Distance for " + valstr + "to" + constr)
        fromRas = "_".join([spp, "PD", valstr])
        fromRas = os.path.join(strWorkspace, fromRas + ".tif")
        if not os.path.isfile(fromRas):
            frmCore = arcpy.SelectLayerByAttribute_management("from_core", "NEW_SELECTION", "CoreID = " + valstr)
            frmDist = sa.PathDistance(frmCore, costRas, elevRas, "#", "#", elevRas,
                      sa.VfTable(os.path.join(strWorkspace, vf)))
            frmDist.save(fromRas)

        toRas = "_".join([spp, "PD", constr])
        toRas = os.path.join(strWorkspace, toRas + ".tif")
        if not os.path.isfile(toRas):
            toCore = arcpy.SelectLayerByAttribute_management("to_core", "NEW_SELECTION", "CoreID = " + constr)
            toDist = sa.PathDistance(toCore, costRas, elevRas, "#", "#", elevRas,
                                     sa.VfTable(os.path.join(strWorkspace, vf)))
            toDist.save(toRas)

        #Get something vaguely like corridors
        print("Now making Corridors")
        create_corridors(fromRas, toRas, spp, maxdist)

On to all the Path Distances
    Creating Path Distance for 606to609
Now making Corridors
    MVerts_Accum_PD_606_to_PD_609.tif saved
Minimum: 2550.8079
[2514.9886146240237, 2714.9886146240237, 2914.9886146240237]
    MVerts_Corridors_PD_606_to_PD_609.tif saved
    Creating Path Distance for 434to61
Now making Corridors
    MVerts_Accum_PD_434_to_PD_61.tif saved
Minimum: 12178.777
[12330.703501953127, 12530.703501953127, 12730.703501953127]
    MVerts_Corridors_PD_434_to_PD_61.tif saved
    Creating Path Distance for 445to434
Now making Corridors
    MVerts_Accum_PD_445_to_PD_434.tif saved
Minimum: 2188.023
[2145.129396728516, 2345.129396728516, 2545.129396728516]
    MVerts_Corridors_PD_445_to_PD_434.tif saved
    Creating Path Distance for 510to7
Now making Corridors
    MVerts_Accum_PD_510_to_PD_7.tif saved
Minimum: 4042.4353
[4035.7027911376954, 4235.702791137695, 4435.702791137695]
    MVerts_Corridors_PD_510_to_PD_7.tif saved
    Creating Path Distance for 515to46478
Now making C

## Clean-up

In [8]:
del curSrch
arcpy.SetLogHistory(True)
arcpy.CheckInExtension("Spatial")

Totes finished


[^1]: Armstrong, D.M., J.P. Fitzgerald, and C.A. Meaney. 2011. Mammals of Colorado, 2nd edition. Denver Museum of Nature and Science and University Press of Colorado, Boulder, CO.